#  Q4 Link prediction


In [1]:
from abc import ABC, abstractmethod
import networkx as nx
import numpy as np
from sklearn.metrics import precision_score, recall_score
from tqdm import tqdm
import random

class LinkPrediction(ABC):
    def __init__(self, graph):
        """
        Constructor

        Parameters
        ----------
        graph : Networkx graph
        """
        self.graph = graph
        self.N = len(graph)

    def neighbors(self, v):
        """
        Return the neighbors list of a node

        Parameters
        ----------
        v : int
            node id

        Return
        ------
        neighbors_list : python list
        """
        neighbors_list = self.graph.neighbors(v)
        return list(neighbors_list)

    @abstractmethod
    def fit(self):
        raise NotImplementedError("Fit must be implemented")

class CommonNeighbors(LinkPrediction):
    def __init__(self, graph):
        super(CommonNeighbors, self).__init__(graph)

    def fit(self, u, v):
        """
        Compute Common Neighbors score for nodes u and v

        Parameters
        ----------
        u, v : int
            Node IDs

        Returns
        -------
        float
            Number of common neighbors
        """
        u_neighbors = set(self.neighbors(u))
        v_neighbors = set(self.neighbors(v))
        return len(u_neighbors.intersection(v_neighbors))

class Jaccard(LinkPrediction):
    def __init__(self, graph):
        super(Jaccard, self).__init__(graph)

    def fit(self, u, v):
        """
        Compute Jaccard coefficient for nodes u and v

        Parameters
        ----------
        u, v : int
            Node IDs

        Returns
        -------
        float
            Jaccard coefficient
        """
        u_neighbors = set(self.neighbors(u))
        v_neighbors = set(self.neighbors(v))
        intersection = len(u_neighbors.intersection(v_neighbors))
        union = len(u_neighbors.union(v_neighbors))
        return 0.0 if union == 0 else intersection / union

class AdamicAdar(LinkPrediction):
    def __init__(self, graph):
        super(AdamicAdar, self).__init__(graph)

    def fit(self, u, v):
        """
        Compute Adamic/Adar score for nodes u and v

        Parameters
        ----------
        u, v : int
            Node IDs

        Returns
        -------
        float
            Adamic/Adar index
        """
        u_neighbors = set(self.neighbors(u))
        v_neighbors = set(self.neighbors(v))
        common_neighbors = u_neighbors.intersection(v_neighbors)

        score = 0.0
        for w in common_neighbors:
            degree = len(self.neighbors(w))
            if degree > 1:  # Avoid log(1) = 0
                score += 1.0 / np.log(degree)
        return score

def evaluate_link_prediction(G, predictor_class, fraction=0.1, k_values=[50, 100, 200, 300, 400]):
    """
    Evaluate link prediction performance

    Parameters
    ----------
    G : networkx.Graph
        Input graph
    predictor_class : class
        Link prediction class to evaluate
    fraction : float
        Fraction of edges to remove
    k_values : list
        List of k values for top-k evaluation

    Returns
    -------
    dict
        Dictionary containing evaluation metrics
    """
    G_working = G.copy()

    num_edges_to_remove = int(fraction * G.number_of_edges())
    edges_to_remove = random.sample(list(G.edges()), num_edges_to_remove)
    G_working.remove_edges_from(edges_to_remove)

    predictor = predictor_class(G_working)

    predictions = []
    nodes = list(G_working.nodes())

    for i in tqdm(range(len(nodes))):
        for j in range(i + 1, len(nodes)):
            u, v = nodes[i], nodes[j]
            if not G_working.has_edge(u, v):
                score = predictor.fit(u, v)
                predictions.append((u, v, score))

    predictions.sort(key=lambda x: x[2], reverse=True)

    results = {}
    for k in k_values:
        top_k_edges = {(u, v) for u, v, _ in predictions[:k]}
        removed_edge_set = set(edges_to_remove)

        correct_predictions = len(top_k_edges.intersection(removed_edge_set))

        precision = correct_predictions / k
        recall = correct_predictions / len(removed_edge_set)
        top_k = correct_predictions / min(k, len(removed_edge_set))

        results[k] = {
            'precision': precision,
            'recall': recall,
            'top_k': top_k
        }

    return results

# Single University

In [10]:

G = nx.read_gml('data/Caltech36.gml')

fractions = [0.05, 0.1, 0.15, 0.2]
k_values = [50, 100, 200, 300, 400]

predictors = [CommonNeighbors, Jaccard, AdamicAdar]

random_number = random.randint(0, 3)



for predictor_class in predictors:
    print(f"\nEvaluating {predictor_class.__name__}")

    fraction = fractions[random_number]
    print(f"\nFraction of edges removed: {fraction}")
    results = evaluate_link_prediction(G, predictor_class, fraction, k_values)

    for k, metrics in results.items():
        print(f"\nk={k}:")
        print(f"Precision@{k}: {metrics['precision']:.3f}")
        print(f"Recall@{k}: {metrics['recall']:.3f}")
        print(f"Top@{k}: {metrics['top_k']:.3f}")


Evaluating CommonNeighbors

Fraction of edges removed: 0.2


100%|██████████| 769/769 [00:03<00:00, 239.92it/s]



k=50:
Precision@50: 0.760
Recall@50: 0.011
Top@50: 0.760

k=100:
Precision@100: 0.740
Recall@100: 0.022
Top@100: 0.740

k=200:
Precision@200: 0.710
Recall@200: 0.043
Top@200: 0.710

k=300:
Precision@300: 0.670
Recall@300: 0.060
Top@300: 0.670

k=400:
Precision@400: 0.645
Recall@400: 0.077
Top@400: 0.645

Evaluating Jaccard

Fraction of edges removed: 0.2


100%|██████████| 769/769 [00:04<00:00, 182.36it/s]



k=50:
Precision@50: 0.420
Recall@50: 0.006
Top@50: 0.420

k=100:
Precision@100: 0.530
Recall@100: 0.016
Top@100: 0.530

k=200:
Precision@200: 0.505
Recall@200: 0.030
Top@200: 0.505

k=300:
Precision@300: 0.520
Recall@300: 0.047
Top@300: 0.520

k=400:
Precision@400: 0.505
Recall@400: 0.061
Top@400: 0.505

Evaluating AdamicAdar

Fraction of edges removed: 0.2


100%|██████████| 769/769 [00:05<00:00, 153.74it/s]



k=50:
Precision@50: 0.820
Recall@50: 0.012
Top@50: 0.820

k=100:
Precision@100: 0.790
Recall@100: 0.024
Top@100: 0.790

k=200:
Precision@200: 0.775
Recall@200: 0.047
Top@200: 0.775

k=300:
Precision@300: 0.740
Recall@300: 0.067
Top@300: 0.740

k=400:
Precision@400: 0.700
Recall@400: 0.084
Top@400: 0.700


# 15 Universities

In [4]:

G = nx.read_gml('data/Caltech36.gml')


graph_files = [
    'data/Caltech36.gml',
    'data/Reed98.gml',
    'data/Simmons81.gml',
    'data/Haverford76.gml',
    'data/Swarthmore42.gml',
    'data/USFCA72.gml',
    'data/Bowdoin47.gml',
    'data/Mich67.gml',
    'data/Amherst41.gml',
    'data/Oberlin44.gml',
    'data/Hamilton46.gml',
    'data/Wellesley22.gml'
]


fractions = [0.05, 0.1, 0.15, 0.2]
k_values = [50, 100, 200, 300, 400]

predictors = [CommonNeighbors, Jaccard, AdamicAdar]

for graph_file in graph_files:
      G = nx.read_gml(graph_file)
      print(f"\nEvaluating for graph: {graph_file}")

      random_number = random.randint(0, 3)
      fraction = fractions[random_number]

      for predictor_class in predictors:
            print(f"\nEvaluating {predictor_class.__name__} for graph: {graph_file}")
            print(f"\nFraction of edges removed: {fraction}")
            results = evaluate_link_prediction(G, predictor_class, fraction, k_values)

            for k, metrics in results.items():
                print(f"\nk={k}:")
                print(f"Precision@{k}: {metrics['precision']:.3f}")
                print(f"Recall@{k}: {metrics['recall']:.3f}")
                print(f"Top@{k}: {metrics['top_k']:.3f}")


Evaluating for graph: data/Caltech36.gml

Evaluating CommonNeighbors for graph: data/Caltech36.gml

Fraction of edges removed: 0.2


100%|██████████| 769/769 [00:02<00:00, 286.11it/s]



k=50:
Precision@50: 0.840
Recall@50: 0.013
Top@50: 0.840

k=100:
Precision@100: 0.810
Recall@100: 0.024
Top@100: 0.810

k=200:
Precision@200: 0.705
Recall@200: 0.042
Top@200: 0.705

k=300:
Precision@300: 0.697
Recall@300: 0.063
Top@300: 0.697

k=400:
Precision@400: 0.670
Recall@400: 0.080
Top@400: 0.670

Evaluating Jaccard for graph: data/Caltech36.gml

Fraction of edges removed: 0.2


100%|██████████| 769/769 [00:05<00:00, 148.73it/s]



k=50:
Precision@50: 0.500
Recall@50: 0.008
Top@50: 0.500

k=100:
Precision@100: 0.560
Recall@100: 0.017
Top@100: 0.560

k=200:
Precision@200: 0.490
Recall@200: 0.029
Top@200: 0.490

k=300:
Precision@300: 0.500
Recall@300: 0.045
Top@300: 0.500

k=400:
Precision@400: 0.477
Recall@400: 0.057
Top@400: 0.477

Evaluating AdamicAdar for graph: data/Caltech36.gml

Fraction of edges removed: 0.2


...


k=50:
Precision@50: 0.560
Recall@50: 0.001
Top@50: 0.560

k=100:
Precision@100: 0.590
Recall@100: 0.003
Top@100: 0.590

k=200:
Precision@200: 0.595
Recall@200: 0.006
Top@200: 0.595

k=300:
Precision@300: 0.563
Recall@300: 0.009
Top@300: 0.563

k=400:
Precision@400: 0.545
Recall@400: 0.011
Top@400: 0.545

Evaluating Jaccard for graph: data/Wellesley22.gml

Fraction of edges removed: 0.2


100%|██████████| 2970/2970 [01:41<00:00, 29.30it/s] 



k=50:
Precision@50: 0.560
Recall@50: 0.001
Top@50: 0.560

k=100:
Precision@100: 0.530
Recall@100: 0.003
Top@100: 0.530

k=200:
Precision@200: 0.555
Recall@200: 0.006
Top@200: 0.555

k=300:
Precision@300: 0.570
Recall@300: 0.009
Top@300: 0.570

k=400:
Precision@400: 0.545
Recall@400: 0.011
Top@400: 0.545

Evaluating AdamicAdar for graph: data/Wellesley22.gml

Fraction of edges removed: 0.2


100%|██████████| 2970/2970 [02:01<00:00, 24.38it/s] 



k=50:
Precision@50: 0.760
Recall@50: 0.002
Top@50: 0.760

k=100:
Precision@100: 0.740
Recall@100: 0.004
Top@100: 0.740

k=200:
Precision@200: 0.705
Recall@200: 0.007
Top@200: 0.705

k=300:
Precision@300: 0.680
Recall@300: 0.011
Top@300: 0.680

k=400:
Precision@400: 0.635
Recall@400: 0.013
Top@400: 0.635
